### 构建卷积神经网络

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets,transforms 
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

### 首先读取数据
- 分别构建训练集和测试集
- DataLoader来迭代取数据

In [9]:
# 定义超参数 
input_size = 28  #输入图像的尺寸28*28
num_classes = 10  #标签的种类数
num_epochs = 2  #训练的总循环周期
batch_size = 64  #一个batch的大小为64张图片

# 加载训练集
train_dataset = datasets.MNIST(root='./data',  
                            train=True,   
                            transform=transforms.ToTensor(),  
                            download=True) 

# 加载测试集
test_dataset = datasets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

# 构建batch数据
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

### 卷积网络模块构建
- 一般卷积层，relu层，池化层可以写成一个单元
- 注意卷积最后结果还是一个特征图，需要把图转换成向量才能继续做分类或者回归任务

In [10]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 第一个卷积单元
        self.conv1 = nn.Sequential(         # 输入大小 28*28*1
            nn.Conv2d(
                in_channels=1,              # 输入特征图的个数
                out_channels=16,            # 卷积核的个数，输出特征图的个数
                kernel_size=5,              # 卷积核大小
                stride=1,                   # 步长
                padding=2,                  # 保证卷积后大小跟原来一样
            ),                              # 输出的特征图为 28*28*16
            nn.ReLU(),                      # relu层
            nn.MaxPool2d(kernel_size=2),    # 进行池化操作（2*2 区域）, 输出14*14*16
        )
        # 第二个卷积单元
        self.conv2 = nn.Sequential(         # 输入 14*14*16
            nn.Conv2d(16, 32, 5, 1, 2),     # 卷积层输出 14*14*32
            nn.ReLU(),                      # relu层
            nn.MaxPool2d(2),                # 池化层输出 7*7*32
        )
        self.out = nn.Linear(32 * 7 * 7, 10)   # 全连接层得到结果

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        # 全连接层之前把矩阵拉成长向量
        # flatten操作，结果为：(batch_size, 32 * 7 * 7)
        x = x.view(x.size(0), -1)          
        output = self.out(x)
        return output

### 准确率作为评估标准

In [11]:
def accuracy(predictions, labels):
    # 取概率最大的种类为预测值
    pred = torch.max(predictions.data, 1)[1] 
    # rights为正确预测的个数，把储存正确标签的数组拉成pred相同的格式
    rights = pred.eq(labels.data.view_as(pred)).sum() 
    return rights, len(labels) 

### 训练网络模型

In [12]:
# 实例化
net = CNN() 
# 交叉熵损失函数
criterion = nn.CrossEntropyLoss() 
# 优化器,用Adam进行优化 （用SGD优化的最后准确率只有0.84）
optimizer = optim.Adam(net.parameters(), lr=0.001) 

#开始训练循环
for epoch in range(num_epochs):
    #当前epoch的结果保存下来
    train_rights = [] 
    
    #针对容器中的每一个batch进行循环
    # enumerate() 遍历，会给出索引和数据  batch_idx--index  data  target--correct labels
    for batch_idx, (data, target) in enumerate(train_loader):  
        net.train()                             
        output = net(data) 
        loss = criterion(output, target) 
        # 将梯度值置零，否则会累加
        optimizer.zero_grad() 
        # 反向传播
        loss.backward() 
        # 更新参数
        optimizer.step() 
        right = accuracy(output, target) 
        # 正确预测的个数存入train_rights
        train_rights.append(right) 

    
        if batch_idx % 100 == 0: 
            # 验证集上不用做反向传播及更新参数，只要计算准确率
            net.eval() 
            val_rights = [] 
            
            for (data, target) in test_loader:
                output = net(data) 
                right = accuracy(output, target) 
                val_rights.append(right)
                
            #准确率计算
            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))

            print('当前epoch: {} [{}/{}]\t损失: {:.6f}\t训练集准确率: {:.2f}%\t测试集准确率: {:.2f}%'.format(
                epoch+1, batch_idx * batch_size, len(train_loader.dataset),loss.data, 
                # 把数据从tensor转化成Ndarray
                100. * train_r[0].numpy() / train_r[1], 
                100. * val_r[0].numpy() / val_r[1]))

当前epoch: 1 [0/60000]	损失: 2.305474	训练集准确率: 9.38%	测试集准确率: 11.23%
当前epoch: 1 [6400/60000]	损失: 0.540366	训练集准确率: 76.50%	测试集准确率: 91.31%
当前epoch: 1 [12800/60000]	损失: 0.311992	训练集准确率: 84.41%	测试集准确率: 94.55%
当前epoch: 1 [19200/60000]	损失: 0.155483	训练集准确率: 87.86%	测试集准确率: 96.08%
当前epoch: 1 [25600/60000]	损失: 0.038801	训练集准确率: 89.79%	测试集准确率: 96.65%
当前epoch: 1 [32000/60000]	损失: 0.077894	训练集准确率: 91.25%	测试集准确率: 97.31%
当前epoch: 1 [38400/60000]	损失: 0.159698	训练集准确率: 92.20%	测试集准确率: 97.57%
当前epoch: 1 [44800/60000]	损失: 0.082892	训练集准确率: 92.94%	测试集准确率: 98.04%
当前epoch: 1 [51200/60000]	损失: 0.223878	训练集准确率: 93.51%	测试集准确率: 98.02%
当前epoch: 1 [57600/60000]	损失: 0.091114	训练集准确率: 93.96%	测试集准确率: 98.17%
当前epoch: 2 [0/60000]	损失: 0.068678	训练集准确率: 96.88%	测试集准确率: 98.18%
当前epoch: 2 [6400/60000]	损失: 0.126177	训练集准确率: 97.88%	测试集准确率: 98.08%
当前epoch: 2 [12800/60000]	损失: 0.061812	训练集准确率: 97.93%	测试集准确率: 97.72%
当前epoch: 2 [19200/60000]	损失: 0.021864	训练集准确率: 97.92%	测试集准确率: 98.15%
当前epoch: 2 [25600/60000]	损失: 0.038583	训练集准确率: 98.00%	测试集准确率